In [1]:
  from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My\ Drive/stylegan-encoder

/content/gdrive/My Drive/stylegan-encoder


In [0]:
import matplotlib.pyplot as plt
import scipy.stats
import numpy.linalg as la
import os
import argparse
import pickle
from tqdm import tqdm
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
from encoder.perceptual_model import PerceptualModel
import tensorflow as tf
import seaborn as sns

In [0]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'  # karras2019stylegan-ffhq-1024x1024.pkl

def split_to_batches(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

src_dir = "./aligned_realpics/LR/"
generated_images_dir = "generated_images/"
dlatent_dir = "latent_representations/"
image_size = 128
lr = 1

ref_images = [os.path.join(src_dir, x) for x in os.listdir(src_dir)]
ref_images = list(filter(os.path.isfile, ref_images))

if len(ref_images) == 0:
    raise Exception('%s is empty' % src_dir)

os.makedirs(generated_images_dir, exist_ok=True)
os.makedirs(dlatent_dir, exist_ok=True)

# Initialize generator and perceptual model
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

In [0]:
rnd = np.random.RandomState(5)
mean_list = []
for i in range(5):
  print(i)
  latents = rnd.randn(100000, 512)
  out = Gs_network.components.mapping.get_output_for(latents, None)
  mean_latent = tf.reduce_mean(out,axis=0)
  mean_list.append(mean_latent[0].eval())

latent_avg = np.mean(mean_list,axis=0)

In [0]:
rnd = np.random.RandomState(5)
latents = rnd.randn(1000, 512)
out = Gs_network.components.mapping.get_output_for(latents, None)
out = out.eval()

In [0]:
latent_avg = np.mean(mean_list,axis=0)
np.save("latent_avg.npy",latent_avg)

In [0]:
dlatent = np.load('latent_representations/chicago2_01.npy')
plt.hist(dlatent.flatten(), bins=500, density=True);
print((np.sum(np.square(np.maximum(0,dlatent)))*0.25+np.sum(np.square(np.minimum(0,dlatent)))*4)/200)
print((np.sum(np.square(np.maximum(0,out)))*0.25+np.sum(np.square(np.minimum(0,out)))*4)/(200*1000))

In [0]:
generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((512, 512))
  
def display_tensor(im):
    im=np.transpose(im,(1,2,0))
    im = (im+1)/2
    im = np.clip(im,0,1)
    plt.figure(figsize = (10,10))
    plt.imshow(im)
    plt.axis('off')
    
def gen_disp(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    img_array = Gs_network.components.synthesis.run(latent_vector)
    img_array = img_array[0]
    display_tensor(img_array)

In [0]:
def rescale(x,a,b):
  x[x < 0] *= a
  x[x >= 0] *= b
  
  return x

In [0]:
mu=[]
sigma=[]
for i in range(512):
  a,b = scipy.stats.norm.fit(rescale(out[:,0,i].flatten(),1,1/5))
  mu.append(a)
  sigma.append(b)

In [0]:
np.save("gaussian_fit.npy",(mu,sigma))

In [0]:
i=np.random.randint(0,512)
sns.distplot((out[:,0,i]-0.8*np.maximum(0,out[:,0,i])).flatten())
# sns.distplot(latents[:,i].flatten())

In [0]:
latent_avg = np.load("latent_avg.npy")

In [0]:
sns.distplot(latent_avg)

In [0]:
gen_disp(np.repeat(np.repeat(latent_avg.mean(), 512, 0), 18, axis=0))

In [0]:
gen_disp(latent_avg)

In [0]:
plt.imshow(np.cov(out[:,0,:].T))

In [0]:
scale_out = np.maximum(0,out)/5+np.minimum(0,out)
import seaborn as sns
sns.distplot(scale_out.flatten(),fit=scipy.stats.norm,kde=False)
mu,sigma=scipy.stats.norm.fit(scale_out.flatten())
np.save("gaussian_fit.npy",(mu,sigma))

In [0]:
mu,sigma=np.load("gaussian_fit.npy")

In [0]:
latents = rescale(rnd.randn(10000, 512)*sigma+mu,1,5)
# latents = np.repeat(latents, 18, axis=0)
sns.distplot(latents.flatten());

In [0]:
rnd = np.random.RandomState(5)
latents = rnd.randn(1000, 512)
out_flat = Gs_network.components.mapping.get_output_for(latents, None)
out_flat=out_flat.eval()
out_flat = out_flat[:,0,:]

In [0]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=1).fit(out_flat)

In [0]:
samples,_ = gmm.sample(1000);
sns.distplot(samples.flatten());

In [0]:
mu,sigma = np.load("gaussian_fit.npy")
latent_avg = np.load("latent_avg.npy")

rnd = np.random.RandomState(10)
z_latent = rnd.randn(1, 512)
latent_out = Gs_network.components.mapping.get_output_for(z_latent, None)
latent_out=latent_out.eval()

latent_out[0:7,:] = latent_avg + (latent_out[0:7,:]-latent_avg)*0.7

gen_disp(latent_out)

In [0]:
mu,sigma = np.load("gaussian_fit.npy")

In [0]:
# mu,sigma = np.load("gaussian_fit.npy")
latent_avg = np.load("latent_avg.npy")

rnd = np.random.RandomState(120)
latents = rnd.randn(18, 512)*sigma+mu
# latents = np.repeat(latents, 18, axis=0)

# latents[0:7,:] = mu + (latents[0:7,:] - mu)*0.7

# latents = 2*mu-latents

latents=rescale(latents,1,5)

gen_disp(latents)

In [12]:
!python encode_images.py ./aligned_realpics/LR/ generated_images/ latent_representations/ --image_size 64 --lr 0

Using TensorFlow backend.
Current Path: /content/gdrive/My Drive/stylegan-encoder
Last python file (encoder/perceptual_model.py) was modified 55.63034 seconds ago
Instructions for updating:
Colocations handled automatically by placer.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 1659, in _create_c_op
    c_op = c_api.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Dimensions must be equal, but are 1024 and 3 for 'BiasAdd' (op: 'BiasAdd') with input shapes: [1,64,64,1024], [3].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "encode_images.py", line 114, in <module>
    main()
  File "encode_images.py", line 69, in main
    perceptual_model.build_perceptual_model(generator.generator_output)
  File "/content/gdrive/My Drive/stylegan-encoder/encoder/perceptual_model.py", line 57, in build_perceptual_model
 